# 09 Major Transit QA/QC

In [1]:
import os
import sys
import yaml
import pickle
import glob
import copy

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

# Remote I/O and parameters

In [4]:
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
root_dir = "D:/github/lfs-cleaning/travel-model-two-networks"
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_07')
output_dir = os.path.join(root_dir, 'data', 'processed', 'version_08')
card_dir = os.path.join(root_dir, 'project_cards')
# lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')
lasso_dir = "Z:/Data/Users/Sijia/MTC/github/Lasso"

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-04-07 17:38:42, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-07 17:38:42, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-07 17:38:42, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-07 17:38:42, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


# Read pickle

In [6]:
version_08_pickle_file_name = os.path.join(input_dir, 'working_scenario_08.pickle')
v_08_scenario = pickle.load(open(version_08_pickle_file_name, 'rb'))

In [7]:
v_08_scenario.road_net.links_df[(v_08_scenario.road_net.links_df.A == 1010070) &
                                (v_08_scenario.road_net.links_df.B == 1012353)]

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,segment_id,ML_useclass,ML_access,ML_egress,tollseg,transit_access,tollbooth,ML_tollbooth,ML_tollseg,id
236835,nan,True,True,30e541ca021182657e868c1eae7e8ad7,1,nan,23rd Street,False,nan,residential,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2b948e69cb2f2636f32fb673595e598e


In [8]:
temp = pd.merge(v_08_scenario.transit_net.feed.trips,
               v_08_scenario.transit_net.feed.frequencies[["trip_id", "start_time"]],
               how = "left",
               on = ["trip_id"])

In [9]:
temp[temp.route_id == "382"][["direction_id", "shape_id", "start_time", "trip_id"]]

,direction_id,shape_id,start_time,trip_id
871,0.0,833,21600.0,12770
872,0.0,833,54000.0,12731
873,0.0,833,36000.0,12699
879,1.0,839,54000.0,12901
880,1.0,839,36000.0,12872
881,1.0,839,21600.0,12822


### check if street rail to street connectors are added

In [10]:
link_df = pd.read_csv("../../data/interim/major_transit_corrected_line_files/rail-to-street-connector-links.csv")

In [11]:
pd.merge(link_df, 
         v_08_scenario.road_net.links_df[
             ["A", "B", "model_link_id", "roadway", "drive_access", "walk_access", "rail_only"]],
        how = "left",
        on = ["A", "B"])

,A,B,Line,model_link_id,roadway,drive_access,walk_access,rail_only
0,1027971,1017359,N,74689,None,1,True,0
1,1027969,1009869,N,26067378,,,,1
2,1027970,1003537,N,26067379,,,,1
3,1028035,1016050,N,74753,None,1,True,0
4,1009896,1028036,N,26067381,,,,1
5,1002433,1027908,L,26067382,,,,1
6,1027826,1002433,L,26067383,,,,1
7,1000377,1015900,M,23293,primary,1,True,0
8,1012446,1027946,M,74231,None,1,True,0
9,1010580,1027905,K,26067386,,,,1


# Create scenario and apply projects

In [12]:
v_09_scenario = Scenario.create_scenario(
    base_scenario= v_08_scenario,
    card_directory = card_dir,
    tags = ["Major Transit"],
    validate_project_cards=False)

2021-04-07 17:43:54, INFO: Creating Scenario
2021-04-07 17:43:54, INFO: Creating Scenario


z:\data\users\sijia\mtc\github\network_wrangler\network_wrangler\projectcard.py:115: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  attribute_dictionary = yaml.load(_yaml)


In [13]:
v_09_scenario.road_net.links_df.model_link_id.max()

26067396

In [14]:
len(v_09_scenario.project_cards)

24

In [15]:
v_09_scenario.apply_all_projects()

2021-04-07 17:50:53, ERROR: Projects SFMTA Route N-JUDAH has move street rail to street connector links as prerequisite project which is missing for the scenario
2021-04-07 17:50:53, ERROR: Projects SFMTA Route N-JUDAH has move street rail to street connector links as prerequisite project which is missing for the scenario
2021-04-07 17:50:53, INFO: Applying VTA Route 22
2021-04-07 17:50:53, INFO: Applying VTA Route 22
2021-04-07 17:50:53, INFO: Applying VTA Route 22
2021-04-07 17:50:53, INFO: Applying VTA Route 22
2021-04-07 17:50:53, INFO: Applying Project to Transit Network: VTA Route 22
2021-04-07 17:50:53, INFO: Applying Project to Transit Network: VTA Route 22
2021-04-07 17:50:54, WARNING: Missing connections from node 2129493 to node 2118093 for the new routing.
2021-04-07 17:50:54, WARNING: Missing connections from node 2129493 to node 2118093 for the new routing.
2021-04-07 17:50:54, INFO: Routing using default graph from node 2129493 to node 2118093 for missing connections.
20

c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


2021-04-07 17:50:54, INFO: Creating a new stop in stops.txt for node ID: 2078170
2021-04-07 17:50:54, INFO: Creating a new stop in stops.txt for node ID: 2078170
2021-04-07 17:50:54, INFO: Creating a new stop in stops.txt for node ID: 2104183
2021-04-07 17:50:54, INFO: Creating a new stop in stops.txt for node ID: 2104183
2021-04-07 17:50:54, INFO: Creating a new stop in stops.txt for node ID: 2015328
2021-04-07 17:50:54, INFO: Creating a new stop in stops.txt for node ID: 2015328
2021-04-07 17:50:56, INFO: Applying VTA Route 22
2021-04-07 17:50:56, INFO: Applying VTA Route 22
2021-04-07 17:50:56, INFO: Applying Project to Transit Network: VTA Route 22
2021-04-07 17:50:56, INFO: Applying Project to Transit Network: VTA Route 22
2021-04-07 17:50:57, WARNING: Missing connections from node 2033565 to node 2103889 for the new routing.
2021-04-07 17:50:57, WARNING: Missing connections from node 2033565 to node 2103889 for the new routing.
2021-04-07 17:50:57, INFO: Routing using default gra

2021-04-07 17:51:00, WARNING: Missing connections from node 1004296 to node 1012711 for the new routing.
2021-04-07 17:51:00, WARNING: Missing connections from node 1004296 to node 1012711 for the new routing.
2021-04-07 17:51:00, INFO: Routing using default graph from node 1004296 to node 1012711 for missing connections.
2021-04-07 17:51:00, INFO: Routing using default graph from node 1004296 to node 1012711 for missing connections.
2021-04-07 17:51:00, WARNING: Missing connections from node 1004311 to node 1017370 for the new routing.
2021-04-07 17:51:00, WARNING: Missing connections from node 1004311 to node 1017370 for the new routing.
2021-04-07 17:51:00, INFO: Routing using default graph from node 1004311 to node 1017370 for missing connections.
2021-04-07 17:51:00, INFO: Routing using default graph from node 1004311 to node 1017370 for missing connections.
2021-04-07 17:51:00, WARNING: Missing connections from node 1022175 to node 1013412 for the new routing.
2021-04-07 17:51:00

2021-04-07 17:51:01, INFO: Creating a new stop in stops.txt for node ID: 1006862
2021-04-07 17:51:01, INFO: Creating a new stop in stops.txt for node ID: 1014552
2021-04-07 17:51:01, INFO: Creating a new stop in stops.txt for node ID: 1014552
2021-04-07 17:51:02, INFO: Applying SFMTA Route N-JUDAH
2021-04-07 17:51:02, INFO: Applying SFMTA Route N-JUDAH
2021-04-07 17:51:03, INFO: Applying Project to Transit Network: SFMTA Route N-JUDAH
2021-04-07 17:51:03, INFO: Applying Project to Transit Network: SFMTA Route N-JUDAH
2021-04-07 17:51:04, WARNING: Missing connections from node 1012711 to node 1004296 for the new routing.
2021-04-07 17:51:04, WARNING: Missing connections from node 1012711 to node 1004296 for the new routing.
2021-04-07 17:51:04, INFO: Routing using default graph from node 1012711 to node 1004296 for missing connections.
2021-04-07 17:51:04, INFO: Routing using default graph from node 1012711 to node 1004296 for missing connections.
2021-04-07 17:51:04, WARNING: Missing c

...

2021-04-07 17:51:19, INFO: Creating a new stop in stops.txt for node ID: 1018891
2021-04-07 17:51:19, INFO: Creating a new stop in stops.txt for node ID: 1018891
2021-04-07 17:51:21, INFO: Applying SFMTA Route KT
2021-04-07 17:51:21, INFO: Applying SFMTA Route KT
2021-04-07 17:51:21, INFO: Applying SFMTA Route KT
2021-04-07 17:51:21, INFO: Applying SFMTA Route KT
2021-04-07 17:51:21, INFO: Applying Project to Transit Network: SFMTA Route KT
2021-04-07 17:51:21, INFO: Applying Project to Transit Network: SFMTA Route KT
2021-04-07 17:51:22, WARNING: Missing connections from node 1015120 to node 1011908 for the new routing.
2021-04-07 17:51:22, WARNING: Missing connections from node 1015120 to node 1011908 for the new routing.
2021-04-07 17:51:22, INFO: Routing using default graph from node 1015120 to node 1011908 for missing connections.
2021-04-07 17:51:22, INFO: Routing using default graph from node 1015120 to node 1011908 for missing connections.
2021-04-07 17:51:22, WARNING: Missing 

2021-04-07 17:51:30, INFO: Routing using default graph from node 1013142 to node 1028011 for missing connections.
2021-04-07 17:51:30, INFO: Routing using default graph from node 1013142 to node 1028011 for missing connections.
2021-04-07 17:51:31, WARNING: Missing connections from node 1013142 to node 1028011 for the new routing.
2021-04-07 17:51:31, WARNING: Missing connections from node 1013142 to node 1028011 for the new routing.
2021-04-07 17:51:31, INFO: Routing using default graph from node 1013142 to node 1028011 for missing connections.
2021-04-07 17:51:31, INFO: Routing using default graph from node 1013142 to node 1028011 for missing connections.
2021-04-07 17:51:31, INFO: Creating a new stop in stops.txt for node ID: 1005725
2021-04-07 17:51:31, INFO: Creating a new stop in stops.txt for node ID: 1005725
2021-04-07 17:51:33, INFO: Applying SFMTA Route CC-MASON
2021-04-07 17:51:33, INFO: Applying SFMTA Route CC-MASON
2021-04-07 17:51:33, INFO: Applying SFMTA Route CC-MASON
2

2021-04-07 17:51:47, INFO: Applying SFMTA Route 9R
2021-04-07 17:51:47, INFO: Applying Project to Transit Network: SFMTA Route 9R
2021-04-07 17:51:47, INFO: Applying Project to Transit Network: SFMTA Route 9R
2021-04-07 17:51:48, INFO: Creating a new stop in stops.txt for node ID: 1010376
2021-04-07 17:51:48, INFO: Creating a new stop in stops.txt for node ID: 1010376
2021-04-07 17:51:49, INFO: Applying SFMTA Route 5R
2021-04-07 17:51:49, INFO: Applying SFMTA Route 5R
2021-04-07 17:51:49, INFO: Applying SFMTA Route 5R
2021-04-07 17:51:49, INFO: Applying SFMTA Route 5R
2021-04-07 17:51:49, INFO: Applying Project to Transit Network: SFMTA Route 5R
2021-04-07 17:51:49, INFO: Applying Project to Transit Network: SFMTA Route 5R
2021-04-07 17:51:51, INFO: Applying SFMTA Route 49
2021-04-07 17:51:51, INFO: Applying SFMTA Route 49
2021-04-07 17:51:51, INFO: Applying SFMTA Route 49
2021-04-07 17:51:51, INFO: Applying SFMTA Route 49
2021-04-07 17:51:51, INFO: Applying Project to Transit Network:

2021-04-07 17:52:21, INFO: Creating a new stop in stops.txt for node ID: 1015493
2021-04-07 17:52:21, INFO: Creating a new stop in stops.txt for node ID: 1015493
2021-04-07 17:52:21, INFO: Applying SFMTA Route 30
2021-04-07 17:52:21, INFO: Applying SFMTA Route 30
2021-04-07 17:52:21, INFO: Applying Project to Transit Network: SFMTA Route 30
2021-04-07 17:52:21, INFO: Applying Project to Transit Network: SFMTA Route 30
2021-04-07 17:52:23, INFO: Creating a new stop in stops.txt for node ID: 1007878
2021-04-07 17:52:23, INFO: Creating a new stop in stops.txt for node ID: 1007878
2021-04-07 17:52:25, INFO: Applying SFMTA Route 29
2021-04-07 17:52:25, INFO: Applying SFMTA Route 29
2021-04-07 17:52:25, INFO: Applying SFMTA Route 29
2021-04-07 17:52:25, INFO: Applying SFMTA Route 29
2021-04-07 17:52:25, INFO: Applying Project to Transit Network: SFMTA Route 29
2021-04-07 17:52:25, INFO: Applying Project to Transit Network: SFMTA Route 29
2021-04-07 17:52:26, WARNING: Missing connections from

2021-04-07 17:52:52, INFO: Applying Project to Transit Network: AC Transit Route 1
2021-04-07 17:52:53, WARNING: Missing connections from node 2576452 to node 2569242 for the new routing.
2021-04-07 17:52:53, WARNING: Missing connections from node 2576452 to node 2569242 for the new routing.
2021-04-07 17:52:53, INFO: Routing using default graph from node 2576452 to node 2569242 for missing connections.
2021-04-07 17:52:53, INFO: Routing using default graph from node 2576452 to node 2569242 for missing connections.
2021-04-07 17:52:53, INFO: Creating a new stop in stops.txt for node ID: 2581780
2021-04-07 17:52:53, INFO: Creating a new stop in stops.txt for node ID: 2581780
2021-04-07 17:52:54, INFO: Applying AC Transit Route 1
2021-04-07 17:52:54, INFO: Applying AC Transit Route 1
2021-04-07 17:52:54, INFO: Applying Project to Transit Network: AC Transit Route 1
2021-04-07 17:52:54, INFO: Applying Project to Transit Network: AC Transit Route 1
2021-04-07 17:52:56, WARNING: Missing con

In [16]:
v_09_scenario.applied_projects

['VTA Route 607 Edits',
 'VTA Route 606 Edits',
 'VTA Route 605 Edits',
 'VTA Route 593 Edits',
 'VTA Route 570 Edits',
 'VTA Route 569 Edits',
 'SFMTA Route 381 Edits',
 'SFMTA Route 376 Edits',
 'SFMTA Route 349 Edits',
 'SFMTA Route 346 Edits',
 'SFMTA Route 336 Edits',
 'SFMTA Route 335 Edits',
 'SFMTA Route 329 Edits',
 'SFMTA Route 328 Edits',
 'SFMTA Route 321 Edits',
 'SFMTA Route 307 Edits',
 'SFMTA Route 306 Edits',
 'SFMTA Route 305 Edits',
 'SFMTA Route 304 Edits',
 'sf bay ferry vallejo',
 'sf bay ferry vallejo missing link',
 'sf bay ferry missing link',
 'sf bay ferry al-ok-sf',
 'SamTrans Route 459 Edits',
 'SamTrans Route 393 Edits',
 'BART Fremont Daly City reverse shape',
 'AC Transit Route 99 Edits',
 'AC Transit Route 40 Edits',
 'AC Transit Route 38 Edits',
 'AC Transit Route 06 Edits',
 'move street rail to street connector links',
 'VTA Route 22',
 'SFMTA Route N-JUDAH',
 'SFMTA Route M-OCEANVIEW',
 'SFMTA Route L-TARAVAL',
 'SFMTA Route KT',
 'SFMTA Route J-CHU

In [17]:
v_09_scenario.transit_net.feed.shapes.shape_id.nunique()

1561

# As transit line file (for CUBE)

In [18]:
standard_transit_net = StandardTransit.fromTransitNetwork(v_09_scenario.transit_net, parameters = parameters)

2021-04-07 17:53:26, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-07 17:53:26, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-07 17:53:26, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-04-07 17:53:26, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


In [19]:
mtc.write_as_cube_lin(standard_transit_net, parameters, outpath = os.path.join(output_dir, "transit.lin"))

2021-04-07 17:53:33, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-07 17:53:33, INFO: Converting GTFS Standard Properties to MTC's Cube Standard


In [20]:
for agency in standard_transit_net.feed.routes.agency_raw_name.unique():
    sub_transit_net = copy.deepcopy(standard_transit_net)
    sub_transit_net.feed.trips = sub_transit_net.feed.trips[sub_transit_net.feed.trips.agency_raw_name == agency]
    mtc.write_as_cube_lin(sub_transit_net, parameters, outpath = os.path.join(output_dir, agency + "_transit.lin"))

2021-04-07 18:01:50, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-07 18:01:50, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-07 18:01:58, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-07 18:01:58, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-07 18:01:59, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-07 18:01:59, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-07 18:02:25, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-07 18:02:25, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-07 18:02:28, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-07 18:02:28, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-07 18:02:31, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-04-07 18:02:31, INFO: Converting GTFS Standard Pr